<a href="https://colab.research.google.com/github/wtsyang/dl-reproducibility-project/blob/BN-without-dropout/cnnTrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
# Mount Google Drive
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)

drive.mount(ROOT)           # we mount the google drive at /content/drive

/content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
from google.colab import files

uploaded = files.upload()

Saving Model.py to Model (1).py


In [0]:
from Model import Model

In [0]:
try:
    from torchsummary import summary
except ModuleNotFoundError:
    !pip install -q torchsummary
    from torchsummary import summary


In [0]:
try:
    import torch
except ModuleNotFoundError:
    from os import path
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

    accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.1-{platform}-linux_x86_64.whl
    import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

In [0]:
try:
    import torchvision
except ModuleNotFoundError:
    !pip install -q torchvision

from torchvision import datasets, transforms

In [0]:
cuda = True
train_batch_size = 256
test_batch_size = 124
best_loss = float("inf")
best_epoch = -1
dataset_path = './cifar10'
gsync_save = True

In [0]:
if gsync_save:
    try:
        import utils
    except ModuleNotFoundError:
        !wget https://raw.githubusercontent.com/StefOe/colab-pytorch-utils/HEAD/utils.py
        import utils

    gsync = utils.GDriveSync()

In [23]:
cuda = cuda and torch.cuda.is_available()
trainset = datasets.CIFAR10(root=dataset_path, train=True, download=True)
train_mean = trainset.data.mean(axis=(0,1,2))/255  # [0.49139968  0.48215841  0.44653091]
train_std = trainset.data.std(axis=(0,1,2))/255  # [0.24703223  0.24348513  0.26158784]
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(train_mean, train_std),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(train_mean, train_std),
])
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}
train_loader = torch.utils.data.DataLoader(datasets.CIFAR10(
    root=dataset_path, train=True, download=True,
    transform=transform_train),
    batch_size=train_batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10(root=dataset_path, train=False, download=True,
    transform=transform_test),
    batch_size=test_batch_size, shuffle=False, **kwargs)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [0]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data))

In [0]:
def test(epoch, best_loss, best_epoch, lr):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)

        output = model(data)
        # sum up batch loss
        test_loss += criterion(output, target).data
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

    test_loss /= len(test_loader.dataset)
    print(
        '\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(test_loader.dataset), 100. * correct /
            len(test_loader.dataset)))
    
    if test_loss < best_loss:
        best_epoch = epoch
        best_loss = test_loss
        torch.save(model, "best_"+str(lr)+".pt")
        try:
          if gsync_save:
              gsync.update_file_to_folder("best_"+str(lr)+".pt")
        except:
          print('Failed to gsync_save.')

        try:
          model_save_name = 'best_'+str(lr)
          path = F"/content/drive/My Drive/dl-reproducibility-project/model/{model_save_name}" 
          torch.save(model.state_dict(), path+'.epoch-{}.pt'.format(epoch))
        except:
          print('Failed to save best model to personal google drive')

    return best_loss, best_epoch, correct

In [26]:
try:
  os.makedirs('/content/drive/My Drive/dl-reproducibility-project/model/')
except:
  print('')

In [27]:
for lr in [0.1,0.05,0.01]:
  print("Training with ", lr)

  
  try: 
    del model
  except:
    print('')
  
  # Maybe it's redundant 
  try:
    del criterion
    del optimizer
    del scheduler
  except:
    print('')
  
  try:
    torch.cuda.empty_cache()
  except:
    print('')
  
  #
  # Chnage Here
  #
  # Choose Model A by basemodel=[True, False, False] and choose B by basemodel=[False, True, False]
  # Choose the modification by modifiedModel=[True, False, False, False]. The first one is the model without any modification.
  # The scecond one is the strided. The third: ConvPool. The last: All CNN.
  model = Model(dropOut=False, BN=True)
  if cuda:
      model.cuda()

  criterion = nn.CrossEntropyLoss()
  optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
  scheduler = optim.lr_scheduler.MultiStepLR(
      optimizer, milestones=[200, 250, 300], gamma=0.1)
  print(model)
  summary(model.model, input_size=(3, 32, 32))
  for epoch in range(350):
      scheduler.step()
      train(epoch)
      best_loss, best_epoch, acc = test(epoch, best_loss, best_epoch, lr)
      if (epoch>100)&(acc.item()>0)&((acc.item()/len(test_loader.dataset))<=0.1):
        print("Stop at acc of test set:",acc.item()/len(test_loader.dataset))
        break
  # if in Google Colab, download your model with this
  #from google.colab import files
  #files.download("best_"+str(lr)+"pt")

Training with  0.1
Model(
  (model): Sequential(
    (0): Conv2d(3, 96, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(96, 192, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): Conv2d(192, 192, kernel_size=(1, 1), stride=(1, 1))
    (12): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU()
    (14): Conv2d(192, 10, kernel_size=(1, 1), stride=(1, 1))
    (15

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:122: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Train Epoch: 0 [0/50000 (0%)]	Loss: 2.355294
Train Epoch: 0 [25600/50000 (51%)]	Loss: 1.382707

Test set: Average loss: 0.0144, Accuracy: 4054/10000 (41%)



Uploading file best_0.1.pt: 100%|██████████| 100/100 [00:02<00:00, 45.42it/s]


Train Epoch: 1 [0/50000 (0%)]	Loss: 1.131070
Train Epoch: 1 [25600/50000 (51%)]	Loss: 1.021504

Test set: Average loss: 0.0146, Accuracy: 4075/10000 (41%)

Train Epoch: 2 [0/50000 (0%)]	Loss: 0.941647
Train Epoch: 2 [25600/50000 (51%)]	Loss: 0.846333

Test set: Average loss: 0.0144, Accuracy: 3736/10000 (37%)



...

Train Epoch: 4 [0/50000 (0%)]	Loss: 0.717550
Train Epoch: 4 [25600/50000 (51%)]	Loss: 0.663859

Test set: Average loss: 0.0141, Accuracy: 4477/10000 (45%)



Uploading file best_0.1.pt: 100%|██████████| 100/100 [00:02<00:00, 44.32it/s]


Train Epoch: 5 [0/50000 (0%)]	Loss: 0.645536
Train Epoch: 5 [25600/50000 (51%)]	Loss: 0.570124

Test set: Average loss: 0.0146, Accuracy: 4122/10000 (41%)

Train Epoch: 6 [0/50000 (0%)]	Loss: 0.487437
Train Epoch: 6 [25600/50000 (51%)]	Loss: 0.591001

Test set: Average loss: 0.0147, Accuracy: 3366/10000 (34%)

Train Epoch: 7 [0/50000 (0%)]	Loss: 0.534618
Train Epoch: 7 [25600/50000 (51%)]	Loss: 0.636732

Test set: Average loss: 0.0151, Accuracy: 3624/10000 (36%)

Train Epoch: 8 [0/50000 (0%)]	Loss: 0.486177
Train Epoch: 8 [25600/50000 (51%)]	Loss: 0.493153

Test set: Average loss: 0.0159, Accuracy: 1698/10000 (17%)

Train Epoch: 9 [0/50000 (0%)]	Loss: 0.581594
Train Epoch: 9 [25600/50000 (51%)]	Loss: 0.520225

Test set: Average loss: 0.0172, Accuracy: 1458/10000 (15%)

Train Epoch: 10 [0/50000 (0%)]	Loss: 0.442648
Train Epoch: 10 [25600/50000 (51%)]	Loss: 0.384440

Test set: Average loss: 0.0175, Accuracy: 1895/10000 (19%)

Train Epoch: 11 [0/50000 (0%)]	Loss: 0.409753
Train Epoch: 11 

Uploading file best_0.05.pt: 100%|██████████| 100/100 [00:01<00:00, 54.24it/s]


Train Epoch: 2 [0/50000 (0%)]	Loss: 0.990870
Train Epoch: 2 [25600/50000 (51%)]	Loss: 0.949684

Test set: Average loss: 0.0139, Accuracy: 3937/10000 (39%)

Train Epoch: 3 [0/50000 (0%)]	Loss: 0.914601
Train Epoch: 3 [25600/50000 (51%)]	Loss: 0.770540

Test set: Average loss: 0.0162, Accuracy: 2722/10000 (27%)

Train Epoch: 4 [0/50000 (0%)]	Loss: 0.765284
Train Epoch: 4 [25600/50000 (51%)]	Loss: 0.781399

Test set: Average loss: 0.0186, Accuracy: 1952/10000 (20%)

Train Epoch: 5 [0/50000 (0%)]	Loss: 0.564087
Train Epoch: 5 [25600/50000 (51%)]	Loss: 0.649235

Test set: Average loss: 0.0156, Accuracy: 3286/10000 (33%)

Train Epoch: 6 [0/50000 (0%)]	Loss: 0.644001
Train Epoch: 6 [25600/50000 (51%)]	Loss: 0.588056

Test set: Average loss: 0.0154, Accuracy: 2854/10000 (29%)

Train Epoch: 7 [0/50000 (0%)]	Loss: 0.543967
Train Epoch: 7 [25600/50000 (51%)]	Loss: 0.601618

Test set: Average loss: 0.0155, Accuracy: 2731/10000 (27%)

Train Epoch: 8 [0/50000 (0%)]	Loss: 0.533233
Train Epoch: 8 [256